# Problem Statement1(Company_Data.csv)

In [1]:
#importing libraries
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

In [2]:
#loading dataset
df  = pd.read_csv("Company_Data.csv")
df

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...
395,12.57,138,108,17,203,128,Good,33,14,Yes,Yes
396,6.14,139,23,3,37,120,Medium,55,11,No,Yes
397,7.41,162,26,12,368,159,Medium,40,18,Yes,Yes
398,5.94,100,79,7,284,95,Bad,50,12,Yes,Yes


# Label Encoding Target Variable

In [3]:
# Calculate the mean sales value
# mean_sales = df['Sales'].mean()
# Create bins to categorize sales into 'Low',High'
# Any value less than or equal to the mean sales is labeled as 'Low',and above the mean is labeled as 'High'
bins = [float('-inf'), df['Sales'].mean(), float('inf')]

# Use the cut function to assign labels to the 'Sales' variable based on the bins
df['SaleCategory'] = pd.cut(df['Sales'], bins=bins, labels=['low','high'])

# Convert the labels to numeric values using LabelEncoder
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['SaleCategory'] = label_encoder.fit_transform(df['SaleCategory'])

# One-Hot Encoding to Independent Variables

In [4]:
# Display the initial dataset
print("Initial Dataset:")
print(df.head())

# Identify categorical columns
categorical_columns = df.select_dtypes(include=['object']).columns

# Apply one-hot encoding to categorical columns
dfen = pd.get_dummies(df, columns=categorical_columns)

# Display the dataset with one-hot encoding
print("\nDataset with One-Hot Encoding:")
dfen

Initial Dataset:
   Sales  CompPrice  Income  Advertising  Population  Price ShelveLoc  Age  \
0   9.50        138      73           11         276    120       Bad   42   
1  11.22        111      48           16         260     83      Good   65   
2  10.06        113      35           10         269     80    Medium   59   
3   7.40        117     100            4         466     97    Medium   55   
4   4.15        141      64            3         340    128       Bad   38   

   Education Urban   US  SaleCategory  
0         17   Yes  Yes             0  
1         10   Yes  Yes             0  
2         12   Yes  Yes             0  
3         14   Yes  Yes             1  
4         13   Yes   No             1  

Dataset with One-Hot Encoding:


,Sales,CompPrice,Income,Advertising,Population,Price,Age,Education,SaleCategory,ShelveLoc_Bad,ShelveLoc_Good,ShelveLoc_Medium,Urban_No,Urban_Yes,US_No,US_Yes
0,9.50,138,73,11,276,120,42,17,0,1,0,0,0,1,0,1
1,11.22,111,48,16,260,83,65,10,0,0,1,0,0,1,0,1
2,10.06,113,35,10,269,80,59,12,0,0,0,1,0,1,0,1
3,7.40,117,100,4,466,97,55,14,1,0,0,1,0,1,0,1
4,4.15,141,64,3,340,128,38,13,1,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,12.57,138,108,17,203,128,33,14,0,0,1,0,0,1,0,1
396,6.14,139,23,3,37,120,55,11,1,0,0,1,1,0,0,1
397,7.41,162,26,12,368,159,40,18,1,0,0,1,0,1,0,1
398,5.94,100,79,7,284,95,50,12,1,1,0,0,0,1,0,1


#  Identify useful features from dataset using feature_importance_

In [5]:
import numpy as np
dt_classifier = DecisionTreeRegressor(random_state=42)
x = dfen.drop(columns = ['Sales','SaleCategory'])
y = dfen['Sales']
dt_classifier.fit(x,y)
print(np.round(dt_classifier.feature_importances_,4))
print(x.columns)

[0.096  0.0804 0.0794 0.042  0.2445 0.0919 0.0282 0.0239 0.2505 0.0463
 0.0096 0.0009 0.0024 0.0039]
Index(['CompPrice', 'Income', 'Advertising', 'Population', 'Price', 'Age',
       'Education', 'ShelveLoc_Bad', 'ShelveLoc_Good', 'ShelveLoc_Medium',
       'Urban_No', 'Urban_Yes', 'US_No', 'US_Yes'],
      dtype='object')


In [6]:
#dropping non-important columns from dataset
dfen = dfen.drop(columns=['Population','Education','ShelveLoc_Bad','Urban_No','Urban_Yes','US_No', 'US_Yes',])

In [7]:
dfen.describe()

,Sales,CompPrice,Income,Advertising,Price,Age,SaleCategory,ShelveLoc_Good,ShelveLoc_Medium
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.00000,400.000000,400.000000
mean,7.496325,124.975000,68.657500,6.635000,115.795000,53.322500,0.50250,0.212500,0.547500
std,2.824115,15.334512,27.986037,6.650364,23.676664,16.200297,0.50062,0.409589,0.498362
min,0.000000,77.000000,21.000000,0.000000,24.000000,25.000000,0.00000,0.000000,0.000000
25%,5.390000,115.000000,42.750000,0.000000,100.000000,39.750000,0.00000,0.000000,0.000000
50%,7.490000,125.000000,69.000000,5.000000,117.000000,54.500000,1.00000,0.000000,1.000000
75%,9.320000,135.000000,91.000000,12.000000,131.000000,66.000000,1.00000,0.000000,1.000000
max,16.270000,175.000000,120.000000,29.000000,191.000000,80.000000,1.00000,1.000000,1.000000


# Decision Tree Regression( BaggingRegressor )

In [8]:
# Define the features (independent variables) and target (dependent variable)
features = dfen.drop(['Sales','SaleCategory'], axis=1)
targetr = dfen['Sales']

# Split the data into training and testing sets
X_trainr, X_testr, y_trainr, y_testr = train_test_split(features, targetr, test_size=0.25)

In [9]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_squared_error

# Create a base model (Decision Tree Regressor)
base_model = DecisionTreeRegressor(random_state=42)

# Create a BaggingRegressor
bagging_regressor = BaggingRegressor(base_model, n_estimators=100, max_samples=0.7,max_features=1.0, random_state=42)

# Train the BaggingRegressor
bagging_regressor.fit(X_trainr, y_trainr)

# Make predictions on the test set
y_pred_bagging = bagging_regressor.predict(X_testr)

# Evaluate the BaggingRegressor (using mean squared error for regression)
mse_bagging = mean_squared_error(y_testr, y_pred_bagging)
print(f'Bagging Mean Squared Error: {mse_bagging:.2f}')

Bagging Mean Squared Error: 2.21


# Decision tree Classification( BaggingClassifier )

In [10]:
# Define the features (independent variables) and target (dependent variable)
features = dfen.drop(['Sales','SaleCategory'], axis=1)
target = dfen['SaleCategory']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.25)

In [11]:
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score

# Create a base model (Decision Tree)
base_model = DecisionTreeClassifier(random_state=42)

# Create a BaggingClassifier
bagging_classifier = BaggingClassifier(base_model, n_estimators=100, random_state=42)

# Train the BaggingClassifier
bagging_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred_bagging = bagging_classifier.predict(X_test)

# Evaluate the BaggingClassifier
accuracy_bagging = accuracy_score(y_test, y_pred_bagging)
print(f'Bagging Accuracy: {accuracy_bagging:.2f}')

Bagging Accuracy: 0.73


In [12]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred_bagging))

              precision    recall  f1-score   support

           0       0.73      0.69      0.71        48
           1       0.73      0.77      0.75        52

    accuracy                           0.73       100
   macro avg       0.73      0.73      0.73       100
weighted avg       0.73      0.73      0.73       100



# finding optimal parameters using gridsearchcv in decision tree classifier

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

# Create a base model (Decision Tree Classifier)
base_model = DecisionTreeClassifier(random_state=42)

# Create a BaggingClassifier
bagging_classifier = BaggingClassifier(base_model, random_state=42)

# Define the hyperparameter grid to search
param_grid = {
    'n_estimators': [10, 50, 100,150, 200],
    'max_samples': [0.3, 0.5, 0.7, 0.9],
    'max_features': [0.5, 0.7, 0.9, 1.0],
    'n_jobs': [-1],
}

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=bagging_classifier, param_grid=param_grid, cv=5, scoring='accuracy')

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and corresponding accuracy
print("Best Parameters: ", grid_search.best_params_)
print("Best Accuracy: {:.2f}".format(grid_search.best_score_))

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy: {:.2f}".format(accuracy))

Best Parameters:  {'max_features': 1.0, 'max_samples': 0.3, 'n_estimators': 150, 'n_jobs': -1}
Best Accuracy: 0.85
Test Accuracy: 0.76


In [14]:
X_train.columns

Index(['CompPrice', 'Income', 'Advertising', 'Price', 'Age', 'ShelveLoc_Good',
       'ShelveLoc_Medium'],
      dtype='object')


# Problem Statement2(Fraudcheck.csv)

# loading dataset

In [15]:
dg = pd.read_csv('Fraud_check.csv')
dg

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO
...,...,...,...,...,...,...
595,YES,Divorced,76340,39492,7,YES
596,YES,Divorced,69967,55369,2,YES
597,NO,Divorced,47334,154058,0,YES
598,YES,Married,98592,180083,17,NO


In [16]:
dg.describe()

,Taxable.Income,City.Population,Work.Experience
count,600.000000,600.000000,600.000000
mean,55208.375000,108747.368333,15.558333
std,26204.827597,49850.075134,8.842147
min,10003.000000,25779.000000,0.000000
25%,32871.500000,66966.750000,8.000000
50%,55074.500000,106493.500000,15.000000
75%,78611.750000,150114.250000,24.000000
max,99619.000000,199778.000000,30.000000


# Create a binary target variable based on the given criteria

In [17]:
# Given that treating those who have taxable_income <= 30000 as "Risky" and others are "Good"
for i in range(len(dg['Taxable.Income'])):
    if (dg['Taxable.Income'].iloc[i] <= 30000):
        dg.loc[i,'Taxable.Income'] = 'Risky'
    else:
        dg.loc[i,'Taxable.Income'] = 'Good'

dg

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,Good,50047,10,YES
1,YES,Divorced,Good,134075,18,YES
2,NO,Married,Good,160205,30,YES
3,YES,Single,Good,193264,15,YES
4,NO,Married,Good,27533,28,NO
...,...,...,...,...,...,...
595,YES,Divorced,Good,39492,7,YES
596,YES,Divorced,Good,55369,2,YES
597,NO,Divorced,Good,154058,0,YES
598,YES,Married,Good,180083,17,NO


In [18]:
#Split data into Independent and Dependent data
x = dg.drop(columns=['Taxable.Income'])
y = dg['Taxable.Income']

# One-Hot Encoding

In [19]:
# Identify categorical columns
categorical_columns = x.select_dtypes(include=['object']).columns

# Apply one-hot encoding to categorical columns
x = pd.get_dummies(x, columns=categorical_columns)

# Display the dataset with one-hot encoding
print("\nDataset with One-Hot Encoding:")
x


Dataset with One-Hot Encoding:


,City.Population,Work.Experience,Undergrad_NO,Undergrad_YES,Marital.Status_Divorced,Marital.Status_Married,Marital.Status_Single,Urban_NO,Urban_YES
0,50047,10,1,0,0,0,1,0,1
1,134075,18,0,1,1,0,0,0,1
2,160205,30,1,0,0,1,0,0,1
3,193264,15,0,1,0,0,1,0,1
4,27533,28,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...
595,39492,7,0,1,1,0,0,0,1
596,55369,2,0,1,1,0,0,0,1
597,154058,0,1,0,1,0,0,0,1
598,180083,17,0,1,0,1,0,1,0


# label encoding

In [20]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,

# Identify useful features from dataset using feature_importance_

In [21]:
import numpy as np
dt_classifier = DecisionTreeRegressor(random_state=42)
dt_classifier.fit(x,y)
print(np.round(dt_classifier.feature_importances_,4))
print(x.columns)

[0.592  0.1812 0.034  0.0269 0.0185 0.0538 0.0349 0.0278 0.0309]
Index(['City.Population', 'Work.Experience', 'Undergrad_NO', 'Undergrad_YES',
       'Marital.Status_Divorced', 'Marital.Status_Married',
       'Marital.Status_Single', 'Urban_NO', 'Urban_YES'],
      dtype='object')


In [22]:
x = x.drop(columns = ['Undergrad_YES','Marital.Status_Divorced','Urban_NO'])

In [23]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.75      0.75      0.75        48
           1       0.77      0.77      0.77        52

    accuracy                           0.76       100
   macro avg       0.76      0.76      0.76       100
weighted avg       0.76      0.76      0.76       100



# DecisionTreeClassifier

In [24]:
#spliting data for training and testing
from sklearn.model_selection import train_test_split
x_train, x_test,y_train,y_test = train_test_split(x,y, test_size=0.25,random_state=40)

In [25]:
#Training Model using DecisionTreeClassifier
model = DecisionTreeClassifier(criterion = 'gini')
model.fit(x_train,y_train)

DecisionTreeClassifier()

In [26]:
y_pred = model.predict(x_test)

In [27]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.77      0.81      0.79       116
           1       0.21      0.18      0.19        34

    accuracy                           0.67       150
   macro avg       0.49      0.49      0.49       150
weighted avg       0.64      0.67      0.65       150



# Gridsearchcv for optimal parameters

In [28]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

# Create a base model (Decision Tree Classifier)
base_model = DecisionTreeClassifier(random_state=42)

# Create a BaggingClassifier
bagging_classifier = BaggingClassifier(base_model, random_state=42)

# Define the hyperparameter grid to search
param_grid = {
    'n_estimators': [10, 50, 100,150, 200],
    'max_samples': [0.3, 0.5, 0.7, 0.9],
    'max_features': [0.5, 0.7, 0.9, 1.0],
    'n_jobs': [-1],
}

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=bagging_classifier, param_grid=param_grid, cv=5, scoring='accuracy')

# Fit the grid search to the data
grid_search.fit(x_train, y_train)

# Print the best parameters and corresponding accuracy
print("Best Parameters: ", grid_search.best_params_)
print("Best Accuracy: {:.2f}".format(grid_search.best_score_))

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy: {:.2f}".format(accuracy))

Best Parameters:  {'max_features': 0.5, 'max_samples': 0.3, 'n_estimators': 100, 'n_jobs': -1}
Best Accuracy: 0.80
Test Accuracy: 0.77


In [29]:
from sklearn.metrics import mean_absolute_error, mean_squared_error,mean_absolute_percentage_error
round(mean_absolute_error(y_test, y_pred),3)

0.227

In [30]:
x_train.columns

Index(['City.Population', 'Work.Experience', 'Undergrad_NO',
       'Marital.Status_Married', 'Marital.Status_Single', 'Urban_YES'],
      dtype='object')

In [31]:
import matplotlib.pyplot as plt
from sklearn import tree
fn = ['City.Population', 'Work.Experience', 'Undergrad_NO',
       'Marital.Status_Married', 'Marital.Status_Single', 'Urban_YES']
cn=['No','Yes']
plt.figure(dpi=1800)
tree.plot_tree(best_model.estimators_[0],class_names = cn,feature_names=fn);